<a href="https://colab.research.google.com/github/limshaocong/analyticsEdge/blob/main/Zero_shot_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-ffbb39df-4dea-15a4-c01e-6e3d900d1a0f)


In [3]:
!pip install transformers==3.1.0

     |████████████████████████████████| 884 kB 15.0 MB/s 
     |████████████████████████████████| 895 kB 73.4 MB/s 
     |████████████████████████████████| 3.0 MB 60.2 MB/s 
     |████████████████████████████████| 1.2 MB 62.1 MB/s 


In [4]:
from transformers import pipeline
import pandas as pd

In [5]:
classifier = pipeline("zero-shot-classification", # transfer learning
                      model = "joeddav/xlm-roberta-large-xnli", # multi-lingual model
                      device = 0) # to utilize GPU

Downloading:   0%|          | 0.00/734 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [6]:
import re
from datetime import datetime
import os.path
from os import path
import requests

In [18]:
def analyse(ticker):
  
  file_path = path + ticker + str("_tweets_2020_cleaned.csv")
  
  df = pd.read_csv(file_path)
  
  df['Text_clean'] = df['Text_clean'].astype(str)

  texts = df["Text_clean"]

  chunks = [texts[x : x + 100] for x in range(0, len(texts), 100)] # split into chunks

  results_list = []

  for idx in range(len(chunks)):

    cp1 = datetime.now()
    
    chunk = chunks[idx].tolist()

    candidate_labels = ["positive", "negative", "neutral"]

    hypothesis_template = "The sentiment of this tweet is {}."

    #dicts = classifier(text, candidate_labels, hypothesis_template = hypothesis_template)
    #result = dicts["labels"][0]
    #results_list.append(result)
    
    list_results_dicts = classifier(chunk, candidate_labels, hypothesis_template = hypothesis_template)

    if type(list_results_dicts) != dict:
      new_list = [i["labels"][0] for i in list_results_dicts] # not robust when chunk size == 1
      results_list.extend(new_list)
    
    else:
      new_list = list_results_dicts["labels"][0]
      results_list.append(new_list)
    
    #results_list.extend(new_list)

    cp2 = datetime.now()

    print(str(idx + 1) + str("/") + str(len(chunks)) + str(" --- ") + str(cp2 - cp1))

  return results_list

In [8]:
from google.colab import files

ticker_path = "https://raw.githubusercontent.com/limshaocong/analyticsEdge/main/TwitterScraper/Tickers_reduced_full.csv"
tickers = pd.read_csv(ticker_path)
tickers_list = tickers["Tickers"].tolist()
tickers_list = tickers_list[126:]

path = "https://raw.githubusercontent.com/limshaocong/analyticsEdge/main//Datasets/Tweet_Analysis/"

In [9]:
tickers_list[:20]

['UGI', 'UNIT', 'VC', 'VERI', 'VIAC', 'VZ', 'WELL', 'WFC', 'WMT', 'Z', 'ZG']

In [ ]:
"""
for ticker in tickers_list:
  
  file_path = path + ticker + str("_tweets_2020_cleaned.csv")
  
  response = requests.get(file_path)

  if response.status_code < 400:
 
    print(ticker + str(" is present"))

    file_path = path + ticker + str("_tweets_2020_cleaned.csv")

    print(file_path)
  
    df = pd.read_csv(file_path)

    results_list = analyse(ticker)

    df["Sentiment"] = results_list

    newfilename = ticker + str("_tweets_2020_tagged.csv")

    df.to_csv(newfilename, index = False)
  
  else:

    print(ticker + str(" is missing"))

    skipped.append(ticker)

    print(skipped)

    continue

"""

In [10]:
skipped = ['CAT',
 'CMCSA',
 'CORE',
 'CRDA',
 'CSCO',
 'CUBI',
 'DISCK',
 'DUK',
 'EBAY',
 'EPR',
 'F',
 'FCNCA',
 'FNB',
 'FOX',
 'FOXA',
 'FWONA',
 'FWONK',
 'MA',
 'MDP',
 'MDT',
 'MMM',
 'MRK',
 'MS',
 'NEE',
 'NKE',
 'NWS',
 'NWSA',
 'OKE',
 'PENN',
 'PFE',
 'PK',
 'POST',
 'PRGO',
 'PRU',
 'QADA',
 'QCOM',
 'QRTEA',
 'RMD',
 'ROG',
 'SBRA',
 'SBUX',
 'SF',
 'SITC',
 'SLM',
 'STAG',
 'STT',
 'STZ',
 'SWM',
 'SYY',
 'T',
 'TAP',
 'TPB',
 'TSLA',
 'TWTR',
 'UA',
 'UAA',
 'AMZN',
 'GOOG',
 'GOOGL',
 'NFLX',
 'IBM',
 'BA',
 'FB']

In [21]:
skipped_1 = ['EBAY', 'F', 'FCNCA', 'FNB', 'FOX', 'FOXA']
skipped_2 = ['OKE', 'PENN']
skipped_3 = ['ROG']
skipped_4 = ['STT', 'STZ', 'SWM', 'SYY', 'T', 'TAP', 'TPB', 'TWTR', 'UA', 'UAA', 'AMZN']
skipped_5 = ['TSLA2', 'TSLA1','FB']

In [ ]:
complete = []

for ticker in skipped_5:
  
  file_path = path + ticker + str("_tweets_2020_cleaned.csv")
  
  response = requests.get(file_path)

  if response.status_code < 400:
      
    print(ticker + str(" is present"))

    file_path = path + ticker + str("_tweets_2020_cleaned.csv")

    print(file_path)
  
    df = pd.read_csv(file_path)

    results_list = analyse(ticker)

    df["Sentiment"] = results_list

    newfilename = ticker + str("_tweets_2020_tagged.csv")

    df.to_csv(newfilename, index = False)

    complete.append(ticker)
  
  else:

    print(ticker + str(" is missing"))

    continue

TSLA2 is present
https://raw.githubusercontent.com/limshaocong/analyticsEdge/main//Datasets/Tweet_Analysis/TSLA2_tweets_2020_cleaned.csv
1/4508 --- 0:00:01.680863
2/4508 --- 0:00:01.608852
3/4508 --- 0:00:01.780503
4/4508 --- 0:00:01.759697
5/4508 --- 0:00:01.736621
6/4508 --- 0:00:01.662958
7/4508 --- 0:00:01.642813
8/4508 --- 0:00:01.776199
9/4508 --- 0:00:01.629594
10/4508 --- 0:00:01.734972
11/4508 --- 0:00:01.668114
12/4508 --- 0:00:01.647105
13/4508 --- 0:00:01.546227
14/4508 --- 0:00:01.722769
15/4508 --- 0:00:01.663014
16/4508 --- 0:00:01.629318
17/4508 --- 0:00:01.545810
18/4508 --- 0:00:01.617325
19/4508 --- 0:00:01.641758
20/4508 --- 0:00:01.737220
21/4508 --- 0:00:01.785274
22/4508 --- 0:00:01.764497
23/4508 --- 0:00:01.619576
24/4508 --- 0:00:01.668299
25/4508 --- 0:00:01.651017
26/4508 --- 0:00:01.744028
27/4508 --- 0:00:01.734811
28/4508 --- 0:00:01.673170
29/4508 --- 0:00:01.521093
30/4508 --- 0:00:01.672599
31/4508 --- 0:00:01.761028
32/4508 --- 0:00:01.759408
33/4508 